# 🌶 Recipe Recommender System

>For anyone - interested in vegetarian, vegan or pescetarian meals

## Installations and importations

In [1]:
%pip install --upgrade pip
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re

## User class

In [3]:
class User:
    #Abstract class/blueprint class for all users.
    def __init__(self, name):
        self.name = name
    #Abstract method to save diet.
    def save_diet(self, diet):
        raise NotImplementedError("This method should be implemented by subclasses.")
    #Abstract method to view diet.
    def view_diet(self):
        raise NotImplementedError("This method should be implemented by subclasses.")


## RegularUser class

In [4]:
class RegularUser(User):
    #Regular user with saved dietary preferences.
    def __init__(self, name):
        super().__init__(name)
        self.diet = None
    #Method to save dietary preference and give confirmation message.
    def save_diet(self, diet):
        self.diet = diet
        print(f"{self.name}, your dietary preference '{self.diet}' has been saved!")
    #Method to update dietary preference.
    def update_diet(self):
        new_diet = input("Enter your dietary preference (vegan, vegetarian, pescetarian): ").strip().lower()
        self.save_diet(new_diet)
    #Method to view dietary preference.
    def view_diet(self):
        return f"{self.name}: {self.diet if self.diet else 'No dietary preference saved'}'"

## AdminUser class

In [5]:

class AdminUser(User):
    #Admin User with password and access to all user data.
    def __init__(self, name, password):
        super().__init__(name)
        self.password = password
        self.users = []

    def add_user(self, user):
        self.users.append(user)

    def view_user_data(self):
        if not self.users:
            print("No user data available.")
        else:
            #Structure to view all user data.
            print("\n--- Overview of User Data in admin database---")
            for user in self.users:
                print(user.view_diet())
            print("------")
            
    def save_diet(self, diet):
        print("Admin users do not save dietary preferences.")

## Recipe class

In [6]:
class Recipe:
    #Class to represent an individual recipe with title, ingredients and instructions.
    def __init__(self, title, ingredients, instructions):
        self.title = title
        self.ingredients = ingredients
        self.instructions = instructions

    def __repr__(self):
        return f"Recipe(title={self.title}, ingredients={self.ingredients[:30]}...)"

## RecipeCategorizer class

In [7]:

class RecipeCategorizer:
    #Categorizes recipes based on dietary preferences.
    def __init__(self, recipes, exclusions_file):
        self.recipes = recipes
        self.exclusions = self.generate_exclusions(exclusions_file)
        
    #This method reads the exclusions file and generates a dictionary of exclusions.
    def generate_exclusions(self, exclusions_file):
        try:
            exclusions_data = pd.read_csv(exclusions_file)
            exclusions = {
                "vegan": exclusions_data[exclusions_data["diet"] == "vegan"]["ingredient_lowercase"].tolist(),
                "vegetarian": exclusions_data[exclusions_data["diet"] == "vegetarian"]["ingredient_lowercase"].tolist(),
                "pescetarian": exclusions_data[exclusions_data["diet"] == "pescetarian"]["ingredient_lowercase"].tolist()
            }
            return exclusions
        except Exception as e:
            print(f"Error loading exclusions: {e}")
            return {"vegan": [], "vegetarian": [], "pescetarian": []}
    #This method cleans and tokenizes the ingredients.
    def clean_and_tokenize(self, ingredients):
        return re.findall(r'\b\w+\b', ingredients.lower())
    
    #This method checks if the recipe is valid based on the ingredients and exclusion list.
    def is_valid_recipe(self, ingredients, exclude_list):
        tokens = self.clean_and_tokenize(ingredients)
        return not any(token in exclude_list for token in tokens)

    #This method categorizes the recies based on the dietary preference.
    
    def categorize(self, diet_type):
        exclude_list = self.exclusions.get(diet_type.lower())
        if not exclude_list:
            raise ValueError(f"Invalid dietary preference: {diet_type}")

        filtered_recipes = self.recipes[
            self.recipes['Ingredients'].apply(lambda x: self.is_valid_recipe(x, exclude_list))
        ]
        return [Recipe(row['Title'], row['Ingredients'], row['Instructions']) for _, row in filtered_recipes.iterrows()]


## RecipeRecommender class

In [8]:
class RecipeRecommender:
    #Provides recipe recommendations based on sorting logic.
    def __init__(self, categorized_recipes):
        self.recipes = categorized_recipes

    #This method uses merge sort to sort the recipes based on the key function.
    def merge_sort(self, recipes, key_func, reverse=False):
        if len(recipes) <= 1:
            return recipes

        mid = len(recipes) // 2
        left = self.merge_sort(recipes[:mid], key_func, reverse)
        right = self.merge_sort(recipes[mid:], key_func, reverse)
        return self._merge(left, right, key_func, reverse)

    def _merge(self, left, right, key_func, reverse):
        sorted_list = []
        while left and right:
            left_key = key_func(left[0])
            right_key = key_func(right[0])
            if (left_key > right_key if reverse else left_key <= right_key):
                sorted_list.append(left.pop(0))
            else:
                sorted_list.append(right.pop(0))
        sorted_list.extend(left)
        sorted_list.extend(right)
        return sorted_list

    #This method recommends recipes based on the number of steps in the instructions.
    def recommend_by_steps(self):
        return self.merge_sort(self.recipes, key_func=lambda r: str(r.instructions).count('.') + str(r.instructions).count('\n'))

    #This method recommends recipes based on the number of ingredients.
    def recommend_by_ingredients(self, user_ingredients):
        def ingredient_match_score(recipe):
            return sum(ing in recipe.ingredients.lower() for ing in user_ingredients.lower().split(', '))

        return self.merge_sort(self.recipes, key_func=ingredient_match_score, reverse=True)



## DataLoader class

In [9]:
#DataLoader class to load data from a file using pandas.
class DataLoader:
    @staticmethod
    def load_data(file_path):
        try:
            return pd.read_csv(file_path)
        except Exception as e:
            print(f"Error loading file: {e}")
            return None

## Main function to run program

In [10]:
#RecipeApp class to run the recipe recommendation app.
class RecipeApp:
    def __init__(self, recipe_file, exclusions_file):
        self.recipe_file = recipe_file
        self.exclusions_file = exclusions_file
        self.data = None
        self.current_user = None
        #Create an admin user with password.
        self.admin = AdminUser("Admin", "admin123")

    #Method to load data from the recipe file.
    def load_data(self):
        self.data = DataLoader.load_data(self.recipe_file)
        if self.data is None:
            print("Failed to load dataset.")
            return False
        return True

    #Method to run the Admin Menu.
    def run_admin_menu(self):
        entered_password = input("Enter admin password: ").strip()
        if entered_password != self.admin.password:
            print("Incorrect password. Access denied.")
            return

        #Specify how the Admin Menu works and looks like.
        while True:
            print("\n🛠Admin Menu:")
            print("1. View user data")
            print("2. Add new user")
            print("3. Switch to Regular User menu")
            print("4. Exit")
            choice = input("Enter your choice: ").strip()

            #Sets conditions for each choice.
            if choice == "1":
                self.admin.view_user_data()
            elif choice == "2":
                user = RegularUser(input("Enter new user's name: ").strip())
                user.update_diet()
                self.admin.add_user(user)
            elif choice == "3":
                self.run_regular_menu()
            elif choice == "4":
                print("Exiting Admin Menu.")
                break
            else:
                print("Invalid choice.")

    #Method to run the Regular User Menu.
    def run_regular_menu(self):
        print("Welcome to the Recipe Recommendation App!🍒")
        name = input("Enter your name: ").strip()
        self.current_user = RegularUser(name)
        self.admin.add_user(self.current_user)  # Automatically add user to admin's list
        while True:
            print("\n👤 Regular User Menu:")
            print("1. Save dietary preference and get recipe recommendations")
            print("2. Switch to Admin Menu🛠")
            print("3. Exit")
            choice = input("Enter your choice: ").strip()

            if choice == "1":
                self.current_user.update_diet()
                self.get_recommendations()
            elif choice == "2":
                self.run_admin_menu()
            elif choice == "3":
                print("Exiting Regular User Menu.")
                break
            else:
                print("Invalid choice.")

    #Method to get recipe recommendations based on the users diet and preferences for sorting.
    def get_recommendations(self):
        categorizer = RecipeCategorizer(self.data, self.exclusions_file)
        try:
            recipes = categorizer.categorize(self.current_user.diet)
            if not recipes:
                print("No recipes found.")
                return
            recommender = RecipeRecommender(recipes)
            print("1. By simplicity in instructions")
            print("2. By ingredients you have in you pantry")
            print("3. I don't care, just show me recipes based on my diet")
            choice = input("Enter choice: ").strip()
            #Sets conditions for each choice, and calls upon methods based on the choice.
            if choice == "1":
                recipes = recommender.recommend_by_steps()
            elif choice == "2":
                recipes = recommender.recommend_by_ingredients(input("Ingredients: "))
            print("\nRecipes:")
            for i, r in enumerate(recipes[:10]):
                print(f"{i+1}. {r.title}")
            self.view_recipe_details(recipes)
        except Exception as e:
            print(e)

    #Method to view recipe details if a user wants to see more details.
    def view_recipe_details(self, recipes):
        while True:
            choice = input("Enter recipe number to view details, 'more' to see more recipes, or 'exit' to go back to the main menu ").strip().lower()
            if choice == 'exit':
                break
            elif choice == 'more':
                for i, r in enumerate(recipes[10:20], 11):
                    print(f"{i}. {r.title}")
            elif choice.isdigit():
                choice = int(choice) - 1
                if 0 <= choice < len(recipes):
                    recipe = recipes[choice]
                    print(f"\nTitle: {recipe.title}\nIngredients: {recipe.ingredients}\nInstructions: {recipe.instructions}\n")
                else:
                    print("Invalid recipe number.")
            else:
                print("Invalid input.")

## Run the program 

In [11]:
#Run the application.
if __name__ == "__main__":
    app = RecipeApp("13k-recipes.csv", "dietary_exclusions.csv")
    app.load_data()
    
    
    #Prepopulate with example users and diets
    example_users = [
        RegularUser("Ellen"),
        RegularUser("Tobias"),
        RegularUser("Beata")
    ]
    
    example_users[0].save_diet("vegan")
    example_users[1].save_diet("vegetarian")
    example_users[2].save_diet("pescetarian")
    
    #Add example users to the admin's list
    for user in example_users:
        app.admin.add_user(user)
    
    app.run_regular_menu()

Ellen, your dietary preference 'vegan' has been saved!
Tobias, your dietary preference 'vegetarian' has been saved!
Beata, your dietary preference 'pescetarian' has been saved!
Welcome to the Recipe Recommendation App!🍒

👤 Regular User Menu:
1. Save dietary preference and get recipe recommendations
2. Switch to Admin Menu🛠
3. Exit
Exiting Regular User Menu.
